In [ ]:
!pip install numpy
!pip install requests
!pip install tqdm
!pip install pandas

In [2]:

from pathlib import Path
import re
import numpy as np  # For generating random delays
import requests
import time
from pathlib import Path
from tqdm.notebook import tqdm
import pandas as pd
import smtplib
from email.mime.text import MIMEText



In [1]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [3]:
df = pd.read_csv('tsv/bangchen1.tsv',sep='\t')

In [104]:

bot='Bot 1 b1'
smtp_server = "smtp.gmail.com"
smtp_port = 587
username = "ADD YOUR EMAIL HERE"  
password = "ADD YOUR APP PASSWORD HERE"  # App password if you use 2FA (you can generate it from your email settings)
def send_request(text, vcn='x2_BoCn_YangJin'):
    url = 'http://ai.tibetbaiyun.com/api/TranslateYuYin'
    payload = {'translatetext': text, 'vcn': vcn}
    try:
        response = requests.post(url, json=payload)
        if response.status_code == 200 and response.json().get('state') == 1:
            return response.json().get('result')
        else:
            return None
    except requests.RequestException:
        return None

def process_tsv_and_download_audio(df, download_folder):
    
    Path(download_folder).mkdir(parents=True, exist_ok=True)

    for _, row in tqdm(df.iterrows(), total=df.shape[0], desc="Downloading Audio"):
        response_url = None
        attempts = 0

        while not response_url and attempts < 5:
            response_url = send_request(row['Text'])
            if not response_url:
                attempts += 1
                time.sleep(5)
  # Wait for 5 seconds before retrying
        if attempts == 5:
            with smtplib.SMTP(smtp_server, smtp_port) as server:
                message = MIMEText(f"{bot}: Man! I am not able to download the audio. Something is wrong with the server")
                message["Subject"] = "Yoo! ALERT! 🚨❗📢"
                message["From"] = username
                message["To"] = "gaychetenzin@gmail.com"
                server.starttls()
                server.login(username, password)
                server.send_message(message)
            break

        if response_url:
            try:
                mp3_response = requests.get(response_url)
                if mp3_response.status_code == 200:
                    file_path = Path(download_folder) / f"{row['ID']}.mp3"
                    with open(file_path, 'wb') as file:
                        file.write(mp3_response.content)
                    with open('checpoint.txt','w') as file:
                        file.write(str(row['ID']))
            except requests.RequestException:
                print(f"Failed to download MP3 for ID {row['ID']}.")

        delay = np.random.lognormal(mean=np.log(10), sigma=0.5)  # Adjust mean and sigma as needed
        delay = np.clip(delay, 3, 15)  # Clipping delay to be between 3 to 15 seconds
        time.sleep(delay)
        #every 6 hours send an email and report the progress

# The function is now ready to be used with the modified delay logic.
# The tsv_file_path and download_folder variables would need to be defined with actual paths before using the function.
try :
    process_tsv_and_download_audio(df, 'Audio')
except Exception as e:
    message=e
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        message = MIMEText(str(e))
        message["Subject"] = f" {bot}: Ahhh shitt !🥲"
        message["From"] = username
        message["To"] = "ADD EMAIL TO SEND ALERTS TO" #use a different email to recieve alerts
        server.starttls()
        server.login(username, password)
        server.send_message(message)   
